In [6]:
 !pip install bs4


In [1]:
import firebase_admin
from firebase_admin import credentials, storage
from firebase_admin import db
import pandas as pd
import os
import dotenv
dotenv.load_dotenv()

True

# Firebase Init

In [2]:
service_account_info ={
    "type": os.getenv('FIREBASE_TYPE'),
    "project_id": os.getenv('FIREBASE_PROJECT_ID'),
    "private_key_id": os.getenv('FIREBASE_PRIVATE_KEY_ID'),
    "private_key": os.getenv('FIREBASE_PRIVATE_KEY'),
    "client_email": os.getenv('FIREBASE_CLIENT_EMAIL'),
    "client_id": os.getenv('FIREBASE_CLIENT_ID'),
    "auth_uri": os.getenv('FIREBASE_AUTH_URL'),
    "token_uri": os.getenv('FIREBASE_TOKEN_URL'),
    "auth_provider_x509_cert_url": os.getenv('FIREBASE_AUTH_PROVIDER_X509_CERT_URL'),
    "client_x509_cert_url": os.getenv('FIREBASE_CLIENT_X509_CERT_URL'),
    "universe_domain": os.getenv('FIREBASE_UNIVERSE_DOMAIN')
  }
  



In [3]:
cred = credentials.Certificate(service_account_info)
firebase_admin.initialize_app(cred,{
    'storageBucket': 'coffee-bot-8ee59.firebasestorage.app',
    'databaseURL': 'https://coffee-bot-8ee59-default-rtdb.firebaseio.com/'})

In [4]:
bucket = storage.bucket()

✅ All images downloaded successfully.


# Upload Data

In [5]:
image_folder_path = './products/images/'

In [6]:
products_collection = db.reference('products')

In [7]:
df = pd.read_json('products/products.jsonl',lines=True)
df.head(2)

,name,category,description,ingredients,price,rating,image_path
0,Cappuccino,Coffee,A rich and creamy cappuccino made with freshly...,"[Espresso, Steamed Milk, Milk Foam]",4.50,4.7,cappuccino.jpg
1,Jumbo Savory Scone,Bakery,"Deliciously flaky and buttery, this jumbo savo...","[Flour, Butter, Cheese, Herbs, Baking Powder, ...",3.25,4.3,SavoryScone.webp


In [8]:
def upload_image(bucket, image_path):
    image_name = image_path.split('/')[-1]
    blob = bucket.blob(f'product_images/{image_name}')
    # Upload image
    blob.upload_from_filename(image_path)
    # Make the image publicly accessible and get its URL
    blob.make_public()
    return blob.public_url

In [9]:
for index, row in df.iterrows():
    print(index, row['name'])
    
    image_path = os.path.join(image_folder_path,row['image_path'])
    
    image_url = upload_image(bucket,image_path)
    product_data = row.to_dict()
    product_data.pop('image_path')
    product_data['image_url']= image_url
    
    # Add to Firestore
    products_collection.push().set(product_data)
    

0 Cappuccino
1 Jumbo Savory Scone
2 Latte
3 Chocolate Chip Biscotti
4 Espresso shot
5 Hazelnut Biscotti
6 Chocolate Croissant
7 Dark chocolate
8 Cranberry Scone
9 Croissant
10 Almond Croissant
11 Ginger Biscotti
12 Oatmeal Scone
13 Ginger Scone
14 Chocolate syrup
15 Hazelnut syrup
16 Carmel syrup
17 Sugar Free Vanilla syrup


In [10]:
!pip install pinecone


  Using cached packaging-24.2-py3-none-any.whl.metadata (3.2 kB)
   ---------------------------------------- 0.0/587.6 kB ? eta -:--:--
   ----------------- ---------------------- 262.1/587.6 kB ? eta -:--:--
   ----------------- ---------------------- 262.1/587.6 kB ? eta -:--:--
   -------------------------------------- 587.6/587.6 kB 693.9 kB/s eta 0:00:00
Using cached packaging-24.2-py3-none-any.whl (65 kB)
  Attempting uninstall: packaging
    Found existing installation: packaging 24.1
    Uninstalling packaging-24.1:
      Successfully uninstalled packaging-24.1
